**Group 22**

Name  | Surname | Email  
---------|-------------------|---------
Julio|Vigueras|20220661@novaims.unl.pt 
Ariel|Pérez|20220662@novaims.unl.pt
Miguelanguel|Mayuare|20220665@novaims.unl.pt
Ayotunde|Aribo|20221012@novaims.unl.pt

# Model handcrafted "C"
---

The previous model "B" improved compared to model "A", it took longer to overfit and had a higher accuracy.

In this model data augmentation is implemented in order to avoid more overfitting.

Data augmentation is atechnique for regularization, which involves generating new training data by applying various random transformations to the existing training samples. By increasing the size and diversity of the training set, data augmentation can improve the model's generalization performance and reduce overfitting. A wide range of transformations can be applied, including rotations, translations, flips, zooms, and color distortions. Data augmentation is particularly effective when the training set is limited or imbalanced, as it can help the model learn more robust and discriminative features from the available data. There have been numerous studies that demonstrate the effectiveness of data augmentation for improving the performance of deep neural networks.

In our dataset it effect was clearly positive, with little overfiting during training.

*Perez, L., & Wang, J. (2017). The effectiveness of data augmentation in image classification using deep learning. arXiv preprint arXiv:1712.04621.*

In [ ]:
# Make the imports
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory

import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

import pathlib
import warnings

warnings.filterwarnings('ignore')

2023-04-08 20:19:08.312978: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 20:19:09.085496: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


First, the previous model is imported in order to make the necessary modifications.

In [ ]:
!wget https://www.dropbox.com/s/n3320qxwdn3rs19/moths.zip?dl=0 -O moths.zip
!unzip moths.zip

Mounted at /content/gdrive/


## Increase of model complexity.

Data augmentation can be implemented by preprocessing images or can be done within the model itself. The following code shows the implementation within the model to be trained.

In [ ]:
def model_C(blocks=4, input_shape=(224, 224, 3)):
    data_augmentation = keras.Sequential([
        # This layer applies a random rotation to the input image at a maximum 
        # angle of 0.2 radians.
        layers.RandomRotation(0.2), 
        # This layer applies a random horizontal or vertical flip to the input image.
        layers.RandomFlip(), 
        # This layer applies a random adjustment to the contrast of the input image 
        # in a maximum range of 0.2.
        layers.RandomContrast(0.2), 
        # This layer applies a random adjustment to the brightness of the input image 
        # over a maximum range of 0.2.
        layers.RandomBrightness(0.2),
        # This layer applies a random zoom to the input image in a maximum range of 0.1.
        layers.RandomZoom(0.1), 
    ])

    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs) 
    # This applies the data augmentation layers defined above to the input images.
    x = layers.Rescaling(1./255)(x)
    for i in range(blocks + 1, blocks + 5):
        # This convolutional layer applies 2^i filters to the input images with a kernel size of 3x3, 
        #padding of "same" and normal Glorot initialization for the weights.
        x = layers.Conv2D(filters=2**i, kernel_size=3,
                          kernel_initializer=initializers.GlorotNormal(seed=123),
                          activation="relu")(x) 
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(30, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = model_C(3)
model.summary()

2023-04-08 20:19:13.428111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 20:19:13.516413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 20:19:13.516563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 16)      0         
                                                             

In [ ]:
# Compile model
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer='adam', 
              metrics=["accuracy"])

In [ ]:
dataset_path = pathlib.Path("moths")
input_shape = (224, 224, 3)
batch_size = 64

In [ ]:
# Split datasets
train_dataset = image_dataset_from_directory(
    dataset_path / "train",
    image_size=input_shape[:2],
    batch_size=batch_size)
validation_dataset = image_dataset_from_directory(
    dataset_path / "valid",
    image_size=input_shape[:2],
    batch_size=batch_size)
test_dataset = image_dataset_from_directory(
    dataset_path / "test",
    image_size=input_shape[:2],
    batch_size=batch_size)

Found 3558 files belonging to 30 classes.
Found 445 files belonging to 30 classes.
Found 408 files belonging to 30 classes.


In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="saved_models/model_handcrafted_C.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30)
]

In [ ]:
# Train model
history = model.fit(
    train_dataset,
    epochs=100,
    batch_size=batch_size,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/100


2023-04-08 20:19:34.486948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3558]
	 [[{{node Placeholder/_4}}]]
2023-04-08 20:19:34.487217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3558]
	 [[{{node Placeholder/_4}}]]
2023-04-08 20:19:35.263325: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-08 20:19:36.334418: I tensorflow/compiler/xla

56/56 [==============================] - ETA: 0s - loss: 3.3430 - accuracy: 0.0509

2023-04-08 20:19:44.935123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [445]
	 [[{{node Placeholder/_4}}]]
2023-04-08 20:19:44.935407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [445]
	 [[{{node Placeholder/_4}}]]


56/56 [==============================] - 11s 95ms/step - loss: 3.3430 - accuracy: 0.0509 - val_loss: 3.1897 - val_accuracy: 0.0764
Epoch 2/100
56/56 [==============================] - 4s 64ms/step - loss: 3.0422 - accuracy: 0.0998 - val_loss: 3.0595 - val_accuracy: 0.1551
Epoch 3/100
56/56 [==============================] - 4s 63ms/step - loss: 2.7062 - accuracy: 0.1922 - val_loss: 2.7353 - val_accuracy: 0.2225
Epoch 4/100
56/56 [==============================] - 4s 64ms/step - loss: 2.2884 - accuracy: 0.3019 - val_loss: 2.2570 - val_accuracy: 0.2921
Epoch 5/100
56/56 [==============================] - 4s 63ms/step - loss: 1.9756 - accuracy: 0.3822 - val_loss: 1.8710 - val_accuracy: 0.3978
Epoch 6/100
56/56 [==============================] - 4s 64ms/step - loss: 1.8341 - accuracy: 0.4323 - val_loss: 1.6378 - val_accuracy: 0.4742
Epoch 7/100
56/56 [==============================] - 4s 64ms/step - loss: 1.6672 - accuracy: 0.4722 - val_loss: 1.7892 - val_accuracy: 0.4719
Epoch 8/100
56/56

In [ ]:
# Visualization
hist_df = pd.DataFrame(history.history)
loss = px.scatter(hist_df['loss'])
val_loss = px.line(hist_df['val_loss'])
accuracy = px.scatter(hist_df['accuracy'])
val_accuracy = px.line(hist_df['val_accuracy'])

fig = make_subplots(cols=2, rows=1, subplot_titles=("Loss", "Accuracy"))
fig.add_trace(loss.data[0], col=1, row=1)
fig.add_trace(val_loss.data[0], col=1, row=1)
fig.add_trace(accuracy.data[0], col=2, row=1)
fig.add_trace(val_accuracy.data[0], col=2, row=1)

fig.show()

![Accuracy and loss](https://www.dropbox.com/s/a4qvflx9xcpiinu/modelc.png?raw=1)

Comparing the plots from the previous model, this is much better because it took longer to overfit.

Below are both models compared with the test set.

In [ ]:
model_A = load_model("saved_models/model_handcrafted_A.keras")
model_B = load_model("saved_models/model_handcrafted_B.keras")
model_C = load_model("saved_models/model_handcrafted_C.keras")

In [ ]:
_, model_A_acc = model_A.evaluate(test_dataset)
_, model_B_acc = model_B.evaluate(test_dataset)
_, model_C_acc = model_C.evaluate(test_dataset)

print(
    f"Model A: {model_A_acc * 100:.2f}% of accuracy\n"
    f"Model B: {model_B_acc * 100:.2f}% of accuracy\n"
    f"Model C: {model_C_acc * 100:.2f}% of accuracy\n"
      )

1/7 [===>..........................] - ETA: 0s - loss: 1.4703 - accuracy: 0.6875

2023-04-08 20:24:40.992308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [408]
	 [[{{node Placeholder/_4}}]]
2023-04-08 20:24:40.992500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [408]
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 0s 18ms/step - loss: 0.5692 - accuracy: 0.8260
Model A: 69.85% of accuracy
Model B: 64.95% of accuracy
Model C: 82.60% of accuracy

